In [2]:
import sys
import re
from typing import *
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
from rbbm_src.labelling_func_src.src.TreeRules import *
from rbbm_src.dc_src.src.classes import parse_rule_to_where_clause,dc_violation_template
import psycopg2
from string import Template

dc_tuple_violation_template=Template("SELECT DISTINCT t1.* FROM $table t1, $table t2 WHERE $dc_desc AND $tuple_desc;")
conn=psycopg2.connect('dbname=holo user=postgres')
cur = conn.cursor()

In [4]:
wrong attribute: work-class

type          _tid_  age workclass      education marital-status      occupation relationship   race   sex hours-per-week native-country income income\n wrong_attr
after_clean   438.0  27  private        hs-grad   married-civ-spouse  craft-repair      husband  white  male             40  united-states  <=50k      NaN  workclass
before_clean  438.0  27  self-emp-inc   hs-grad   married-civ-spouse  craft-repair      husband  white  male             40  united-states  <=50k      NaN  workclass
ground_truth  438.0  27  self-emp-inc   hs-grad   married-civ-spouse  craft-repair      husband  white  male             40  united-states  <=50k      NaN  workclass


In [5]:
complaint_example = pd.read_sql('select * from adult500 where _tid_=438', conn)
# cur.fetchall()
complaint_t = complaint_example.to_dict('records')[0]

In [6]:
complaint_t

{'_tid_': 438,
 'age': '27',
 'workclass': 'self-emp-inc',
 'education': 'hs-grad',
 'marital-status': 'married-civ-spouse',
 'occupation': 'craft-repair',
 'relationship': 'husband',
 'race': 'white',
 'sex': 'male',
 'hours-per-week': '40',
 'native-country': 'united-states',
 'income': '<=50k'}

In [7]:
# dc_violation_template=Template("SELECT t1.* FROM $table t1 WHERE EXISTS (SELECT t2.* FROM $table AS t2 WHERE ($dc_desc));")

In [8]:
responsibilities = {'t1&t2&EQ(t1.education,t2.education)&EQ(t1.marital-status,t2.marital-status)&IQ(t1.relationship,t2.relationship)&EQ(t1.sex,t2.sex)&IQ(t1.workclass,t2.workclass)': [0.25], 
 't1&t2&EQ(t1.education,t2.education)&EQ(t1.hours-per-week,t2.hours-per-week)&IQ(t1.race,t2.race)&IQ(t1.workclass,t2.workclass)': [0.25], 
 't1&t2&EQ(t1.education,t2.education)&EQ(t1.marital-status,t2.marital-status)&EQ(t1.workclass,t2.workclass)&IQ(t1.native-country,t2.native-country)': [-1], 
 't1&t2&EQ(t1.education,t2.education)&IQ(t1.native-country,t2.native-country)&EQ(t1.relationship,t2.relationship)&IQ(t1.workclass,t2.workclass)': [0.25], 
 't1&t2&EQ(t1.occupation,t2.occupation)&EQ(t1.hours-per-week,t2.hours-per-week)&IQ(t1.race,t2.race)&IQ(t1.workclass,t2.workclass)': [0.2], 
 't1&t2&EQ(t1.education,t2.education)&EQ(t1.occupation,t2.occupation)&IQ(t1.race,t2.race)&IQ(t1.workclass,t2.workclass)': [0.25],
 't1&t2&EQ(t1.hours-per-week,t2.hours-per-week)&IQ(t1.native-country,t2.native-country)&IQ(t1.income,t2.income)&EQ(t1.relationship,t2.relationship)&IQ(t1.workclass,t2.workclass)': [0.2]
}
# run_11012022082516
res_tuples = [(k,v[0]) for k,v in responsibilities.items()]


In [9]:
res_tuples = sorted(res_tuples, key=lambda kv:(kv[1], kv[0]), reverse=True)

In [10]:
ops = re.compile(r'IQ|EQ')   

In [11]:
def parse_dc_to_tree_rule(dc_text):
    # input:a dc raw text
    # output: a TreeRule object
    # each predicate comes with 2 nodes:
    #  1.predicate node: describing the predicate text
    #  2.label node: a left child of the predicate node from 1 that says clean
    #  3.set right node of node from 1 to the next predicate. if no predicate is left
    #    then set the right node to dirty
    # support EQ and IQ only so far
    res = []

    predicates = dc_text.split('&')
    root_predicate=predicates[2]
    if(ops.search(root_predicate).group()=='EQ'):
        sign='=='
    else:
        sign='!='
    root_node= PredicateNode(pred=DCAttrPredicate(pred=root_predicate, operator=sign))
    root_left_child= LabelNode(label=CLEAN)
    root_node.left=root_left_child
    parent=root_node
    
    for pred in predicates[3:]:
        if(ops.search(pred).group()=='EQ'):
            sign='=='
        else:
            sign='!='
        cur_node = PredicateNode(pred=DCAttrPredicate(pred=pred, operator=sign))
        root_left_child = LabelNode(label=CLEAN)
        cur_node.left=root_left_child
        parent.right=cur_node
        parent=cur_node
    parent.right=LabelNode(label=DIRTY)
    return TreeRule(rtype='dc', root=root_node)

In [12]:
def find_tuples_in_violation(t_interest, conn, dc_text, target_table):
    predicates = dc_text.split('&')
#     clause = parse_rule_to_where_clause(dc_text)
    constants=[]
    for pred in predicates[2:]:
        attr = re.search(r't[1|2]\.([-\w]+)', pred).group(1)
        constants.append(f't2.\"{attr}\"=\'{t_interest[attr]}\'')
    constants_clause = ' AND '.join(constants)
    r_q  = dc_tuple_violation_template.substitute(table=target_table, dc_desc=parse_rule_to_where_clause(dc_text),
                                           tuple_desc=constants_clause)
    print(r_q)
#     cur.execute(r_q)
#     raw_tuples=cur.fetchall()
#     return [dict(zip(cols, t)) for t in raw_tuples]
    return pd.read_sql(r_q, conn).to_dict('records')
#     return r_q

In [13]:
def gen_repaired_tree_rule(t_interest, desired_label, t_in_violation, target_attribute, tree_rule):
    # given a tuple of interest, the desired label (clean/dirty)
    # a tuple of violation, and a target attribute you want to fix
    # the rule with, return a modified tree rule object that return the 
    # desired label for the pair of tuples
    
    # Step #1, traverse through the given tree rule using the tuple pair (t_interest, t_in_violation)
    pair_to_eval_dict = {'t1':t_interest, 't2':t_in_violation}
    parent, end_node = tree_rule.evaluate(pair_to_eval_dict)
    cur_label = end_node.label
    # Step #2, add a new branch based on the target_attribute and its value in t_interest
    # It is possible the value between t_interest and t_in_violation has the same value in 
    # target_attribute, which wouldnt help
    new_branch = PredicateNode(pred=DCConstPredicate(f"EQ(t1.{target_attribute},'{t_interest[target_attribute]}')"))
    new_branch.right=LabelNode(label=desired_label)
    new_branch.left=LabelNode(label=cur_label)
    # Holoclean constant assignment example EQ(t1.HospitalOwner,'proprietary')
    if(parent.right is end_node):
        parent.right=new_branch
    else:
        parent.left=new_branch
        
    return tree_rule


In [14]:
def fix_violation(t_interest, desired_label, t_in_violation, tree_rule):
    unusable_attrs=['_tid_']
    # check "unavailable nodes", a node is unavailable when
    # the node contains equal operator
    cur_node_parent=None
    cur_node = tree_rule.root
    queue = deque([tree_rule.root])
    while(queue):
        cur_node = queue.popleft()
        if(isinstance(cur_node,PredicateNode)):
            if(cur_node.pred.operator=='=='):
                unusable_attrs.append(cur_node.pred.attr)
        if(cur_node.left):
            queue.append(cur_node.left)
        if(cur_node.right):
            queue.append(cur_node.right)
#         cur_node_parent=cur_node
#         cur_node=cur_node.right
        
    available_attrs=list(set(set(list(t_interest)).difference(set(unusable_attrs))))
    if(not available_attrs):
        return None
    repaired_tree_rule=gen_repaired_tree_rule(t_interest, desired_label, t_in_violation, 
                                              available_attrs[0], tree_rule)
#         if(gen_repair_predicate(t_interest, t_in_violation, attr))
    return repaired_tree_rule

In [15]:
res_tuples

[('t1&t2&EQ(t1.education,t2.education)&IQ(t1.native-country,t2.native-country)&EQ(t1.relationship,t2.relationship)&IQ(t1.workclass,t2.workclass)',
  0.25),
 ('t1&t2&EQ(t1.education,t2.education)&EQ(t1.occupation,t2.occupation)&IQ(t1.race,t2.race)&IQ(t1.workclass,t2.workclass)',
  0.25),
 ('t1&t2&EQ(t1.education,t2.education)&EQ(t1.marital-status,t2.marital-status)&IQ(t1.relationship,t2.relationship)&EQ(t1.sex,t2.sex)&IQ(t1.workclass,t2.workclass)',
  0.25),
 ('t1&t2&EQ(t1.education,t2.education)&EQ(t1.hours-per-week,t2.hours-per-week)&IQ(t1.race,t2.race)&IQ(t1.workclass,t2.workclass)',
  0.25),
 ('t1&t2&EQ(t1.occupation,t2.occupation)&EQ(t1.hours-per-week,t2.hours-per-week)&IQ(t1.race,t2.race)&IQ(t1.workclass,t2.workclass)',
  0.2),
 ('t1&t2&EQ(t1.hours-per-week,t2.hours-per-week)&IQ(t1.native-country,t2.native-country)&IQ(t1.income,t2.income)&EQ(t1.relationship,t2.relationship)&IQ(t1.workclass,t2.workclass)',
  0.2),
 ('t1&t2&EQ(t1.education,t2.education)&EQ(t1.marital-status,t2.marit

In [16]:
# for r in res_tuples[:1]:
treerule = parse_dc_to_tree_rule(res_tuples[0][0])

In [17]:
print(str(treerule))

dc-attr-pred-==EQ(t1.education,t2.education)
	0
	dc-attr-pred-!=IQ(t1.native-country,t2.native-country)
		0
		dc-attr-pred-==EQ(t1.relationship,t2.relationship)
			0
			dc-attr-pred-!=IQ(t1.workclass,t2.workclass)
				0
				1


In [18]:
# test_rule = 't1&t2&EQ(t1.nationality,t2.nationality)&IQ(t1.workclass,t2.workclass)'
# treerule = parse_dc_to_tree_rule(test_rule)
# test_pair_dict={'t1':{'nationality':'china' ,'workclass':'middle'},
#                 't2':{'nationality':'germany' ,'workclass':'high'}}

# test_pair_dict1={'t1':{'a':1 ,'b':2, 'c': 2, 'd': 10},
#                  't2':{'a':1 ,'b':2, 'c': 3, 'd': 9}
#                 }

# test_pair_dict2={'t1':{'a':1 ,'b':2, 'c': 2, 'd': 10},
#                  't2':{'a':1 ,'b':2, 'c': 2, 'd': 10}
#                 }
# treerule.evaluate(test_pair_dict1)
# treerule.evaluate(test_pair_dict2)

In [19]:
complaint_tuple = complaint_t
tuples_inviolation = find_tuples_in_violation(complaint_tuple, conn, res_tuples[0][0], 'adult500')

SELECT DISTINCT t1.* FROM adult500 t1, adult500 t2 WHERE t1."education"=t2."education" AND t1."native-country"!=t2."native-country" AND t1."relationship"=t2."relationship" AND t1."workclass"!=t2."workclass" AND t2."education"='hs-grad' AND t2."native-country"='united-states' AND t2."relationship"='husband' AND t2."workclass"='self-emp-inc';


In [20]:
tuples_inviolation

[{'_tid_': 17,
  'age': '72',
  'workclass': '?',
  'education': 'hs-grad',
  'marital-status': 'married-civ-spouse',
  'occupation': '?',
  'relationship': 'husband',
  'race': 'white',
  'sex': 'male',
  'hours-per-week': '40',
  'native-country': 'cuba',
  'income': '<=50k'},
 {'_tid_': 75,
  'age': '43',
  'workclass': 'private',
  'education': 'hs-grad',
  'marital-status': 'married-civ-spouse',
  'occupation': 'other-service',
  'relationship': 'husband',
  'race': 'white',
  'sex': 'male',
  'hours-per-week': '60',
  'native-country': 'nicaragua',
  'income': '<=50k'},
 {'_tid_': 151,
  'age': '80',
  'workclass': '?',
  'education': 'hs-grad',
  'marital-status': 'married-civ-spouse',
  'occupation': '?',
  'relationship': 'husband',
  'race': 'white',
  'sex': 'male',
  'hours-per-week': '8',
  'native-country': 'canada',
  'income': '<=50k'},
 {'_tid_': 339,
  'age': '36',
  'workclass': 'private',
  'education': 'hs-grad',
  'marital-status': 'married-civ-spouse',
  'occupat

In [21]:
# # {'_tid_': 17,
#   'age': '72',
#   'workclass': '?',
#   'education': 'hs-grad',
#   'marital-status': 'married-civ-spouse',
#   'occupation': '?',
#   'relationship': 'husband',
#   'race': 'white',
#   'sex': 'male',
#   'hours-per-week': '40',
#   'native-country': 'cuba',
#   'income': '<=50k'}

In [22]:
complaint_t

{'_tid_': 438,
 'age': '27',
 'workclass': 'self-emp-inc',
 'education': 'hs-grad',
 'marital-status': 'married-civ-spouse',
 'occupation': 'craft-repair',
 'relationship': 'husband',
 'race': 'white',
 'sex': 'male',
 'hours-per-week': '40',
 'native-country': 'united-states',
 'income': '<=50k'}

In [23]:
fix_violation(complaint_tuple, CLEAN, tuples_inviolation[0], treerule)

dc-attr-pred-==EQ(t1.education,t2.education)
	0
	dc-attr-pred-!=IQ(t1.native-country,t2.native-country)
		0
		dc-attr-pred-==EQ(t1.relationship,t2.relationship)
			0
			dc-attr-pred-!=IQ(t1.workclass,t2.workclass)
				0
				dc-const-pred-EQ(t1.income,'<=50k')
					1
					0

In [24]:
# Algorithm to generate repairs:
# Input: A rule to repair D, the desired label Y

In [25]:
treerule.serialize()

["t1&t2&EQ(t1.education,t2.education)&IQ(t1.native-country,t2.native-country)&EQ(t1.relationship,t2.relationship)&IQ(t1.workclass,t2.workclass)&IQ(t1.income,'<=50k')"]

In [26]:
# new_left=PredicateNode(pred=DCAttrPredicate(pred='EQ(t1.hours-per-week,t2.hours-per-week)&IQ(t1.native-country,t2.native-country)',
#                                    operator='=='))
# new_left.left=LabelNode(label=DIRTY)
# new_left.right=LabelNode(label=CLEAN)
# treerule.root.left=new_left

In [27]:
treerule.serialize()

["t1&t2&EQ(t1.education,t2.education)&IQ(t1.native-country,t2.native-country)&EQ(t1.relationship,t2.relationship)&IQ(t1.workclass,t2.workclass)&IQ(t1.income,'<=50k')"]

In [28]:
print(str(treerule))

dc-attr-pred-==EQ(t1.education,t2.education)
	0
	dc-attr-pred-!=IQ(t1.native-country,t2.native-country)
		0
		dc-attr-pred-==EQ(t1.relationship,t2.relationship)
			0
			dc-attr-pred-!=IQ(t1.workclass,t2.workclass)
				0
				dc-const-pred-EQ(t1.income,'<=50k')
					1
					0


In [29]:
fix_violation(complaint_tuple, CLEAN, tuples_inviolation[1], treerule)

dc-attr-pred-==EQ(t1.education,t2.education)
	0
	dc-attr-pred-!=IQ(t1.native-country,t2.native-country)
		0
		dc-attr-pred-==EQ(t1.relationship,t2.relationship)
			0
			dc-attr-pred-!=IQ(t1.workclass,t2.workclass)
				0
				dc-const-pred-EQ(t1.income,'<=50k')
					1
					dc-const-pred-EQ(t1.hours-per-week,'40')
						0
						0

In [30]:
df = pd.DataFrame({'a':[1,2,1,2], 'b':[3,4,3,5], 'c':[3,4,3,5]})

In [31]:
df

,a,b,c
0,1,3,3
1,2,4,4
2,1,3,3
3,2,5,5


In [32]:
df.sort_values(by=['a','b'])

,a,b,c
0,1,3,3
2,1,3,3
1,2,4,4
3,2,5,5
